## 1. Libraries

In [ ]:
# basic
import math
import numpy as np

# Matplotlib
import matplotlib.pyplot as plt

# opencv
import cv2

# tensorflow
import tensorflow as tf

# keras
import keras
# Keras Base Layer Class
from keras.layers.core import Layer
import keras.backend as K
from keras.utils import np_utils
from keras.models import Model
# Keras Layers
# 1. Convolution Layers
from keras.layers import Conv2D
# 2. Pooling Layers
from keras.layers import MaxPool2D, AveragePooling2D, GlobalAveragePooling2D
# 3. Regularization Layers
from keras.layers import Dropout
# 4. Core Layers
from keras.layers import Input, Dense 
# 4. Merging Layers
from keras.layers import concatenate
# 5. Reshaping Layers
from keras.layers import Flatten
# from keras.layers import Conv2D, MaxPool2D,  \
#     Dropout, Dense, Input, concatenate,      \
#     GlobalAveragePooling2D, AveragePooling2D,\
#     Flatten
# Keras Callbacks 
from keras.callbacks import LearningRateScheduler
# Keras Optimizers
from keras.optimizers import SGD, RMSprop, Adam



# Import Dataset
from keras.datasets import cifar10

## 2. Load Dataset

In [ ]:
(x_train, y_train), (x_valid, y_valid) = cifar10.load_data()

## 3. Data Preprocessing

In [ ]:
# Resize training images
x_train = np.array([cv2.resize(img, (224,224)) for img in x_train[:,:,:,:]])
x_valid = np.array([cv2.resize(img, (224,224)) for img in x_valid[:,:,:,:]])

* Check shape of train & test datasets

In [ ]:
print('Number of training images:',x_train.shape)
print('Number of testing images: ',x_valid.shape)

* Plot some images from train dataset 

In [ ]:
class_names = ['airplane','automobile','bird','cat','deer','dog',
               'frog','horse','ship','truck']

* Check random input and target_label for random image

In [ ]:
plt.xlabel(class_names[y_train[5][0]])
plt.imshow(x_train[5])
plt.show()

In [ ]:
#plot some images from train dataset 
plt.figure(figsize=(10,10))
for i in range(9):
    plt.subplot(3,3,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x_train[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[y_train[i][0]])
plt.show()

#### 3.1 Transform targets to keras compatible format

In [ ]:
num_classes = 10

y_train = np_utils.to_categorical(y_train, num_classes)
y_valid = np_utils.to_categorical(y_valid, num_classes)

#### 3.2 Rescale the images

In [ ]:
# x_train = x_train.astype('float32')
# x_valid = x_valid.astype('float32')

x_train = x_train / 255.0
x_valid = x_valid / 255.0

## 4. Deep Learning Architecture

### 4.1 Design of an Inception Block

In [ ]:
def inception_module(x,
                     filters_1x1,
                     filters_3x3_reduce,
                     filters_3x3,
                     filters_5x5_reduce,
                     filters_5x5,
                     filters_pool_proj,
                     name=None):
    
    conv_1x1 = Conv2D(filters_1x1, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    
    conv_3x3 = Conv2D(filters_3x3_reduce, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_3x3 = Conv2D(filters_3x3, (3, 3), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_3x3)

    conv_5x5 = Conv2D(filters_5x5_reduce, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_5x5 = Conv2D(filters_5x5, (5, 5), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_5x5)

    pool_proj = MaxPool2D((3, 3), strides=(1, 1), padding='same')(x)
    pool_proj = Conv2D(filters_pool_proj, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(pool_proj)

    output = concatenate([conv_1x1, conv_3x3, conv_5x5, pool_proj], axis=3, name=name)
    
    return output

#### Define Initializers

In [ ]:
# The Glorot uniform initializer, also called Xavier uniform initializer
kernel_init = keras.initializers.glorot_uniform()
bias_init = keras.initializers.Constant(value=0.2)

### 4.2 GoogLeNet Architecture

In [ ]:
input_layer = Input(shape=(224, 224, 3))

x = Conv2D(64, (7, 7), padding='same', strides=(2, 2), activation='relu', name='conv_1_7x7/2', kernel_initializer=kernel_init, bias_initializer=bias_init)(input_layer)
x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_1_3x3/2')(x)
x = Conv2D(64, (1, 1), padding='same', strides=(1, 1), activation='relu', name='conv_2a_3x3/1')(x)
x = Conv2D(192, (3, 3), padding='same', strides=(1, 1), activation='relu', name='conv_2b_3x3/1')(x)
x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_2_3x3/2')(x)

x = inception_module(x,
                     filters_1x1=64,
                     filters_3x3_reduce=96,
                     filters_3x3=128,
                     filters_5x5_reduce=16,
                     filters_5x5=32,
                     filters_pool_proj=32,
                     name='inception_3a')

x = inception_module(x,
                     filters_1x1=128,
                     filters_3x3_reduce=128,
                     filters_3x3=192,
                     filters_5x5_reduce=32,
                     filters_5x5=96,
                     filters_pool_proj=64,
                     name='inception_3b')

x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_3_3x3/2')(x)

x = inception_module(x,
                     filters_1x1=192,
                     filters_3x3_reduce=96,
                     filters_3x3=208,
                     filters_5x5_reduce=16,
                     filters_5x5=48,
                     filters_pool_proj=64,
                     name='inception_4a')


x1 = AveragePooling2D((5, 5), strides=3)(x)
x1 = Conv2D(128, (1, 1), padding='same', activation='relu')(x1)
x1 = Flatten()(x1)
x1 = Dense(1024, activation='relu')(x1)
x1 = Dropout(0.7)(x1)
x1 = Dense(10, activation='softmax', name='auxilliary_output_1')(x1)

x = inception_module(x,
                     filters_1x1=160,
                     filters_3x3_reduce=112,
                     filters_3x3=224,
                     filters_5x5_reduce=24,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4b')

x = inception_module(x,
                     filters_1x1=128,
                     filters_3x3_reduce=128,
                     filters_3x3=256,
                     filters_5x5_reduce=24,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4c')

x = inception_module(x,
                     filters_1x1=112,
                     filters_3x3_reduce=144,
                     filters_3x3=288,
                     filters_5x5_reduce=32,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4d')


x2 = AveragePooling2D((5, 5), strides=3)(x)
x2 = Conv2D(128, (1, 1), padding='same', activation='relu')(x2)
x2 = Flatten()(x2)
x2 = Dense(1024, activation='relu')(x2)
x2 = Dropout(0.7)(x2)
x2 = Dense(10, activation='softmax', name='auxilliary_output_2')(x2)

x = inception_module(x,
                     filters_1x1=256,
                     filters_3x3_reduce=160,
                     filters_3x3=320,
                     filters_5x5_reduce=32,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_4e')

x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_4_3x3/2')(x)

x = inception_module(x,
                     filters_1x1=256,
                     filters_3x3_reduce=160,
                     filters_3x3=320,
                     filters_5x5_reduce=32,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_5a')

x = inception_module(x,
                     filters_1x1=384,
                     filters_3x3_reduce=192,
                     filters_3x3=384,
                     filters_5x5_reduce=48,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_5b')

x = GlobalAveragePooling2D(name='avg_pool_5_3x3/1')(x)

x = Dropout(0.4)(x)

x = Dense(10, activation='softmax', name='output')(x)

* Define Model

In [ ]:
model = Model(input_layer, [x, x1, x2], name='inception_v1')

* Keras Model Summary

In [ ]:
model.summary()

#### Define below

1. Loss function for each output layer
2. Weightage assigned to that output layer
3. Optimization function, which is modified to include a weight decay after every 8 epochs Evaluation metric

In [ ]:
epochs = 2
initial_lrate = 0.01

def decay(epoch, steps=100):
    initial_lrate = 0.01
    drop = 0.96
    epochs_drop = 8
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

sgd = SGD(lr=initial_lrate, momentum=0.9, nesterov=False)

lr_sc = LearningRateScheduler(decay, verbose=1)

model.compile(loss=['categorical_crossentropy', 'categorical_crossentropy', 'categorical_crossentropy'], loss_weights=[1, 0.3, 0.3], optimizer=sgd, metrics=['accuracy'])

## 5. Train Model

In [ ]:
history = model.fit(x_train, [y_train, y_train, y_train], validation_data=(x_valid, [y_valid, y_valid, y_valid]), epochs=epochs, batch_size=256, callbacks=[lr_sc])